Downloading and installing necessary packages

In [1]:
!pip install osm-flex
!pip install osmosis
!pip install rasterstats
!pip install rioxarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.7/71.7 kB 6.9 MB/s eta 0:00:00
Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl (363.4 MB)
Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (13.8 MB)
Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl (211.5

In [2]:
# Loading necessary data packages
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
from rasterstats import zonal_stats
from shapely.geometry import Point
from shapely.geometry import LineString

# import sys
# sys.path.append('') #'your-path-to/osm-flex/src'

import osm_flex
import osm_flex.download as dl
import osm_flex.extract as ex
import osm_flex.config
import osm_flex.clip as cp

osm_flex.enable_logs()

1. Using the OSM Flex package to download the Road Network for the Study Area

In [3]:
# Download the Spain country file from download.geofabrik.de
# downloads requested file only if necessary, and returns save path
iso3 = 'ESP'
path_esp_dump = dl.get_country_geofabrik(iso3)
print(f'Saved as {path_esp_dump}')

INFO:osm_flex.download:Download file: /root/osm/osm_bpf/spain-latest.osm.pbf


Saved as /root/osm/osm_bpf/spain-latest.osm.pbf


In [4]:
# available wrapper categories:
osm_flex.config.DICT_CIS_OSM.keys()

dict_keys(['education', 'healthcare', 'water', 'telecom', 'road', 'main_road', 'rail', 'air', 'gas', 'oil', 'power', 'wastewater', 'food', 'buildings'])

In [5]:
# Convert the osm_flex CI dictionary into a Pandas DataFrame for easier exploration of the available CI keys (categories).
DICT_CIS_OSM = pd.DataFrame(osm_flex.config.DICT_CIS_OSM)

# Display the first few rows of the DataFrame to preview its structure and the CI keys it contains.
DICT_CIS_OSM.head()

,education,healthcare,water,telecom,road,main_road,rail,air,gas,oil,power,wastewater,food,buildings
osm_keys,"[amenity, building, name]","[amenity, building, healthcare, name]","[man_made, pump, pipeline, emergency, name]","[man_made, tower_type, telecom, communication_...","[highway, name, maxspeed, lanes, surface]","[highway, name, maxspeed, lanes, surface]","[railway, name, gauge, electrified, voltage]","[aeroway, name]","[man_made, pipeline, utility, name]","[pipeline, man_made, amenity, name]","[power, voltage, utility, name]","[reservoir_type, man_made, utility, natural, n...","[shop, name]","[building, amenity, name]"
osm_query,building='school' or amenity='school' or\n ...,amenity='hospital' or healthcare='hospital' or...,man_made='water_well' or man_made='water_works...,tower_type='communication' or man_made='mast' ...,"highway in ('motorway', 'motorway_link', 'trun...","highway in ('primary', 'primary_link', 'second...",railway='rail' or railway='tram' or\n ...,aeroway='aerodrome',(man_made='pipeline' and substance='gas') or\n...,(pipeline='substation' and substance='oil') or...,power='line' or power='cable' or\n ...,reservoir_type='sewage' or\n ...,shop='supermarket' or shop='greengrocer' or\n ...,building='yes' or building='house' or \n ...


In [6]:
# Optionally, check the signature of the extraction function
? ex.extract_cis

In [ ]:
gdf_esp_mainroad = ex.extract_cis(path_esp_dump, 'main_road')

# Drop rows where geometry type is not LineString
gdf_esp_mainroad = gdf_esp_mainroad[gdf_esp_mainroad.geometry.geom_type == 'LineString']

INFO:osm_flex.extract:query is finished, lets start the loop
extract points: 0it [00:32, ?it/s]
INFO:osm_flex.extract:query is finished, lets start the loop
extract multipolygons:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
fig, ax = plt.subplots()
gdf_esp_mainroad.plot(ax=ax, linewidth=0.5)
fig.suptitle('Main roads in Spain')
plt.show()

In [ ]:
gdf_esp_mainroad.to_csv('Spain_main_roads.csv')
Spain_Roadnetwork = pd.DataFrame(gdf_esp_mainroad)

In [ ]:
Spain_Roadnetwork.head()

In [ ]:
# Assign DataFrame for easier handling
df_Spain = Spain_Roadnetwork

# Columns of interest
columns = ['maxspeed', 'lanes', 'surface']

# Calculate completeness (non-null percentage)
completeness = df_Spain[columns].notnull().mean() * 100

# Plot
plt.figure(figsize=(8, 5))
completeness.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Road Attribute Completeness (%)')
plt.ylabel('Percentage')
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Annotate bars with values
for idx, val in enumerate (completeness):
    plt.text(idx, val + 1, f'{val:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()


Specific Regions

In [ ]:
# Obtain the region polygons
admin_1_esp = cp.get_admin1_shapes('ESP')
admin_1_esp.keys()

In [ ]:
Malaga_shp = admin_1_esp['Málaga']
Malaga_shp

Barcelona_shp = admin_1_esp['Barcelona']
Barcelona_shp

In [ ]:
!rm /root/osm/poly/temp_shp.poly

In [ ]:
# Clip the region polygon:
cp.clip_from_shapes([Malaga_shp],
                    osmpbf_output=osm_flex.config.OSM_DATA_DIR.joinpath('Malaga.osm.pbf'),
                    osmpbf_clip_from=path_esp_dump,
                    kernel='osmosis', overwrite=True)

# Clip the region polygon:
cp.clip_from_shapes([Barcelona_shp],
                    osmpbf_output=osm_flex.config.OSM_DATA_DIR.joinpath('Barcelona.osm.pbf'),
                    osmpbf_clip_from=path_esp_dump,
                    kernel='osmosis', overwrite=True)

In [ ]:
Malaga_Roadnetwork = gdf_esp_mainroad[gdf_esp_mainroad.intersects(Malaga_shp)]
fig, ax = plt.subplots(figsize=(12, 9))
Malaga_Roadnetwork.plot(ax=ax, linewidth=0.5)
fig.suptitle('Main roads in Malaga')
plt.show()

Barcelona_Roadnetwork = gdf_esp_mainroad[gdf_esp_mainroad.intersects(Barcelona_shp)]
fig, ax = plt.subplots(figsize=(12, 9))
Barcelona_Roadnetwork.plot(ax=ax, linewidth=0.5)
fig.suptitle('Main roads in Barcelona')
plt.show()

2. Malaga

In [ ]:
# Assign DataFrame for easier handling
df_Malaga = Malaga_Roadnetwork

# Columns of interest
columns = ['maxspeed', 'lanes', 'surface']

# Calculate completeness (non-null percentage)
completeness = df_Malaga[columns].notnull().mean() * 100

# Plot
plt.figure(figsize=(8, 5))
completeness.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Road Attribute Completeness (%)')
plt.ylabel('Percentage')
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Annotate bars with values
for idx, val in enumerate (completeness):
    plt.text(idx, val + 1, f'{val:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
df_Malaga['Centroid'] = df_Malaga.geometry.centroid
df_Malaga['Midpoint'] = df_Malaga.geometry.apply(lambda geom: geom.interpolate(0.5, normalized=True))
df_Malaga.to_csv('Malaga_main_roads.csv')

In [ ]:
def calculate_sinuosity(geometry):
    # Calculate the actual road distance (length of the LineString)
    road_distance = geometry.length

    # Calculate the Euclidean distance (straight line distance between the first and last points)
    start_point = Point(geometry.coords[0])
    end_point = Point(geometry.coords[-1])
    euclidean_distance = start_point.distance(end_point)

    # Avoid division by zero by adding a small value to the denominator
    sinuosity = road_distance / (euclidean_distance + 1e-100)  # Added 1e-100 to avoid ZeroDivisionError

    return sinuosity

# Apply the function to each geometry in the GeoDataFrame
df_Malaga['sinuosity'] = df['geometry'].apply(calculate_sinuosity)

df_Malaga

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

# Plot road network first with a specific color and zorder
df_Malaga.plot(ax=ax, linewidth=0.5, color='blue', zorder=1)

# Plot centroids (assumed to be in df['Centroid']) on top with a smaller size, color, and transparency
df_Malaga['Centroid'].plot(ax=ax, marker='o', color='red', markersize=0.05, alpha=0.7, zorder=2)

# Add title
fig.suptitle('Main roads in Malaga with Centroids')
plt.show()


In [ ]:
df_Malaga

In [ ]:
CLC_Spain = '/content/CLC_Spain.tif'
CLC_Spain_raster = rxr.open_rasterio(CLC_Spain)  # Open the raster file with rioxarray

# Get the CRS from the opened raster
raster_crs = CLC_Spain_raster.rio.crs

# Ensure roads are in a projected CRS for accurate buffering (meters)
roads = df_Malaga.to_crs(raster_crs)  # Use the raster's CRS for reprojection

# Run zonal stats
stats = zonal_stats(roads['geometry'], CLC_Spain, stats=['majority'], categorical=True)

# Add majority land cover type to roads GeoDataFrame
roads['clc_majority'] = [s['majority'] for s in stats]

In [ ]:
Slope_Spain = '/content/DEM_Malaga.tif'
Slope_Spain_raster = rxr.open_rasterio(Slope_Spain)  # Open the raster file with rioxarray

# Get the CRS from the opened raster
raster_crs = Slope_Spain_raster.rio.crs

# Ensure roads are in a projected CRS for accurate buffering (meters)
roads = df.to_crs(raster_crs)  # Use the raster's CRS for reprojection

# Run zonal stats
stats = zonal_stats(roads['geometry'], Slope_Spain, stats=['mean'], categorical=True)

# Add majority land cover type to roads GeoDataFrame
roads['Average_Slope'] = [s['mean'] for s in stats]

In [ ]:
roads
# Buffer the lines to create polygons (e.g., 5 meters wide)
# roads['geometry'] = roads.geometry.buffer(5)

In [ ]:
roads.to_csv('Slope.csv')

2.2. Barcelona

In [ ]:
import requests
import os

# region identifier
region = "europe/spain/andalucia"

# URL and save path
url = "https://download.geofabrik.de/europe/spain/andalucia-latest.osm.pbf"
# Specify the directory where the file should be saved
save_path = "/root/osm/osm_bpf/"
#The filename will be automatically appended

# Download using the correct region identifier
path_and_dump = dl.get_region_geofabrik(region, save_path=save_path)
print(f'Saved as {path_and_dump}')

In [ ]:
gdf_and_mainroad = ex.extract_cis(path_and_dump, 'main_road')

In [ ]:
fig, ax = plt.subplots()
gdf_and_mainroad.plot(ax=ax, linewidth=0.5)
fig.suptitle('Main roads in Andalusia')
plt.show()

In [ ]:
gdf_and_mainroad.head()
gdf_and_mainroad.to_csv('Andalusia_main_roads.csv')
Andalusia_Roadnetwork = pd.DataFrame(gdf_and_mainroad)

In [ ]:
Andalusia_Roadnetwork.head()

In [ ]:
# Assign DataFrame for easier handling
df = Andalusia_Roadnetwork

# Columns of interest
columns = ['maxspeed', 'lanes', 'surface']

# Calculate completeness (non-null percentage)
completeness = df[columns].notnull().mean() * 100

# Plot
plt.figure(figsize=(8, 5))
completeness.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Road Attribute Completeness (%)')
plt.ylabel('Percentage')
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Annotate bars with values
for idx, val in enumerate (completeness):
    plt.text(idx, val + 1, f'{val:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
from shapely.geometry import LineString
import math

def calculate_sinuosity(coordinates):
    # Create a LineString object
    line = LineString(coordinates)

    # Calculate the actual road distance (length of the LineString)
    road_distance = line.length

    # Calculate the Euclidean distance (straight line distance between the first and last points)
    start_point = coordinates[0]
    end_point = coordinates[-1]
    euclidean_distance = math.sqrt((end_point[0] - start_point[0])**2 + (end_point[1] - start_point[1])**2)

    # Calculate the sinuosity
    sinuosity = road_distance / euclidean_distance

    return sinuosity

# Example usage with a list of coordinates
coordinates = [(-5.7814557, 43.4445928), (-5.7812748, 43.4447708), (-5.7811805, 43.4448446)]

# Calculate the sinuosity for the example coordinates
sinuosity = calculate_sinuosity(coordinates)

print(f"Sinuosity: {sinuosity}")